In [1]:

%env XLA_PYTHON_CLIENT_PREALLOCATE=false
# OR , 
# 80% max prealloc below 
# %env XLA_PYTHON_CLIENT_MEM_FRACTION=0.8

env: XLA_PYTHON_CLIENT_PREALLOCATE=false


In [2]:
# import os
# os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'   # or 'False' — both work
# os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = str(0.8) 

In [3]:
import jax 
import flax
import jax.numpy as jnp
from flax import nnx as nn

class MultiHeadSelfAttention(nn.Module):
    """
    RTX 3060 optimised multi-head self attention module.
    Works best with ampere gpus 
    Functionality:
        - Uses fused QKV projections for better performance 
        - Has mixed precision softmax for improved numerical stability
    """
    def __init__(self, 
                 d_model:int, 
                 num_heads:int, 
                 rngs:nn.Rngs, 
                 dropout_rate:float=0.0,
                 dtype:jnp.dtype=jnp.bfloat16 ):
        super().__init__()
        # ensure tensor dimensions align properly 
        if d_model % num_heads != 0: raise ValueError(f"d_model ({d_model}) must be divisible by num_heads ({num_heads})")
        self.d_model = d_model 
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.dtype=dtype
        
        # scale factor for dot product attention : scale = 1/sqrt(d_k) 
        # normalises the variance of the dot product to prevent it from growing too large,
        # which can lead to vanishing gradients during training
        self.scale = jax.lax.rsqrt(jnp.array(self.d_k, dtype=self.dtype))

        # FUSED PROJECTION: Computes Q, K, V in single matrix multiplication 
        # matrix shape: (d_model, 3 * d_model) to produce concatenated QKV
        # this is more efficient than separate projections for Q, K, V , 
        # since we can fuse a single GEMM operation which is highly optimized on modern hardware
        self.qkv_proj = nn.Linear(
            in_features=self.d_model,
            out_features=3 * self.d_model,
            use_bias=False, # Bias in QKV is generally unnecessary and slows down compute
            dtype=self.dtype,
            param_dtype=self.dtype, 
            rngs=rngs
        )   

        # output projection to combine multi-head outputs back to d_model dimensions
        self.out_proj = nn.Linear(
            in_features=self.d_model, 
            out_features=self.d_model, 
            use_bias=False,
            dtype=self.dtype,
            param_dtype=self.dtype,
            rngs=rngs
        )
        self.dropout = nn.Dropout(rate=dropout_rate, rngs=rngs)

    # forward pass for multi-head self attention
    def __call__(self, x:jax.Array, mask:jax.Array|None=None, deterministic:bool=False) -> jax.Array:
        """
        x: Input tensor of shape (batch_size, seq_length, d_model)
        mask: Optional boolean mask of shape (batch_size, 1, seq_length, seq_length) 
              for masking out padding tokens or future tokens in decoder
        deterministic: If True, disable dropout
        """
        B, L, D = x.shape
        assert D == self.d_model, f"Input feature dimension ({D}) must match model dimension ({self.d_model})"

        # fused projection 
        qkv = self.qkv_proj(x) # shape: (B, L, 3 * d_model)

        # reshape and split into Q, K, V
        # Target shape before split : (B, L, 3, num_heads, d_k)
        qkv = qkv.reshape(B, L, 3, self.num_heads, self.d_k) 

        # transpose to isolate Q, K, V across 3rd dimension 
        # shape : (3, B, num_heads, L, d_k)
        qkv = jnp.transpose(qkv, (2,0,3,1,4)) # shape : (3, B, num_heads, L, d_K)
        q, k, v = qkv[0], qkv[1], qkv[2] 

        # Scaled dot product attention 
        # Einsum handles the batched multi head multiplication
        # Q @ K^T. 
        logits = jnp.einsum('bhqd, bhkd -> bhqk', q, k) * self.scale

        # apply mask if provided, casual masking for decoder 
        if mask is not None:
            # we use a very large negative number to drive the softmax probability to zero 
            # dtype must match logits 
            neg_inf = jnp.array(-1e9, dtype=logits.dtype)
            logits = jnp.where(mask, logits, neg_inf)
        
        # for numerical stability we cast to fp32 for softmax 
        # then back to bfloat16 
        attention_weights = jax.nn.softmax(logits.astype(jnp.float32), axis=-1).astype(self.dtype)
        
        # Apply stochastic dropout on attention weights
        attention_weights = self.dropout(attention_weights, deterministic=deterministic)

        # multiplying attention weights to V 
        # v : (B, num_heads, L, d_k) 
        # attention_weights : (B, num_heads, L, L)
        # attention_weights @ v^T shape: (B, num_heads, L, d_k) 
        context = jnp.einsum('bhqk, bhkd -> bhqd',attention_weights, v) 

        # recombining heads and flattening the last two dimensions 
        context = jnp.transpose(context, (0,2,1,3)).reshape(B, L, self.d_model)

        # final linear projections to get maximum context
        return self.out_proj(context)

In [4]:
# testing out Multi head attention 
def test_mha():
    # static architecture parameters 
    B = 32 # batch_size, 
    L = 128 # max_seq_len 
    d_model=512 # paper replica
    n_heads=8  # paper replica 

    # random state maneger 
    rng = nn.Rngs(744)

    # initiate the model 
    mha = MultiHeadSelfAttention(
        d_model=d_model, 
        num_heads=n_heads, 
        rngs=rng,
        dtype=jnp.bfloat16 # Maximize RTX 3060 throughput
    )

    # dummy data simulating embedding layer output 
    key = jax.random.PRNGKey(seed=0) 
    dummy_x = jax.random.normal(key, (B, L, d_model), dtype=jnp.bfloat16) 

    # dummy mask to test decoder attention
    # Brodcast across batch and heads 
    # Lower triangular matrix of boolean, (true allows attention , false masks it) 
    # shape : (1, 1, L, L) 
    causal_mask = jnp.tril(jnp.ones((1, 1, L, L), dtype=jnp.bool_))

    # forward pass 
    print("--- Multi-Head Attention Forward Pass ---")
    print(f"Input shape (B, L, d_model): {dummy_x.shape} | dtype: {dummy_x.dtype}")
    print(f"Mask shape: {causal_mask.shape}")
    output = mha(dummy_x, mask=causal_mask)
    
    print(f"Output shape (B, L, d_model): {output.shape} | dtype: {output.dtype}")
    print(f"Output device: {output.device}")
    
    # Assertions to ensure strict mathematical invariants
    assert output.shape == dummy_x.shape, "Output geometry must match input geometry"
    assert output.dtype == jnp.bfloat16, "Precision leaked during compute"
    print("All architectural invariants passed."); return mha


In [5]:
model=test_mha()

--- Multi-Head Attention Forward Pass ---
Input shape (B, L, d_model): (32, 128, 512) | dtype: bfloat16
Mask shape: (1, 1, 128, 128)
Output shape (B, L, d_model): (32, 128, 512) | dtype: bfloat16
Output device: cuda:0
All architectural invariants passed.


In [6]:
nn.display(model)

In [7]:
model.qkv_proj.kernel.get_value().shape

(512, 1536)

In [8]:
from typing import Any


class PositionWiseFFN(nn.Module): 
    """
    Position Wise feed forward network
    Applies two linear transformations and a relu activation in between them 
    """
    def __init__(self, d_model:int, d_ffn:int, rngs:nn.Rngs, dtype:jnp.dtype = jnp.bfloat16 ) -> None:
        super().__init__()
        self.ll1 = nn.Linear(in_features=d_model, out_features=d_ffn, dtype=dtype, param_dtype=dtype, rngs=rngs) # expansion layer
        self.ll2 = nn.Linear(in_features=d_ffn, out_features=d_model, dtype=dtype,param_dtype=dtype, rngs=rngs ) # contraction layer

    def __call__(self, x:jax.Array) -> jax.Array:
        # original paper utilises ReLU. Modern variants often use GeLU or SwiGLU, but ReLU is highly efficient .
        return self.ll2(jax.nn.relu(self.ll1(x))) # Layer 1 -> ReLU -> Layer 2 -> Output
    

class EncoderBlock(nn.Module):
    """   
    Single pre layer-norm transformer encoder block, 
    
    Features:
        - Multi Head Self Attention : Queries across all other tokens in a sentence 
        - FFN : Feed forward expands dimension by ~ 3x for more spatial pattern recognition. Then reduce dims to normal
        - Residual Highways : Implemented residual connections across high computation blocks for smooth gradient flow in large networks
    
    """
    def __init__(self, d_model:int, n_heads:int, d_ffn:int, dropout_rate:float, rngs:nn.Rngs, dtype:jnp.dtype=jnp.bfloat16) -> None:
        super().__init__()
        self.mha = MultiHeadSelfAttention(d_model=d_model, num_heads=n_heads, rngs=rngs, dtype=dtype) 
        self.ffn = PositionWiseFFN(d_model=d_model, d_ffn=d_ffn, rngs=rngs, dtype=dtype)

        # pre Layer-Norm : Norms are applied BEFORE the sublayers
        self.ln1 = nn.LayerNorm(num_features=d_model, dtype=dtype, param_dtype=dtype,rngs=rngs)
        self.ln2 = nn.LayerNorm(num_features=d_model, dtype=dtype, param_dtype=dtype, rngs=rngs) 

        # Stochastic regularisation 
        self.dropout1 = nn.Dropout(rate=dropout_rate, rngs=rngs)
        self.dropout2 = nn.Dropout(rate=dropout_rate, rngs=rngs)

    
    def __call__(self, x:jax.Array, mask:jax.Array|None=None, deterministic:bool=False) -> jax.Array:
        """ Put deterministic=True during eval/val/inference to disable dropout """
        # sublayer 1: Multi Head Self Attention (pre-LN and residual connection) 
        norm_x = self.ln1(x); attn_out = self.mha(norm_x, mask=mask); attn_out = self.dropout1(attn_out, deterministic=deterministic)
        x = x + attn_out # residual connection

        # sublayer 2: Positional wise feed forward network 
        norm_x = self.ln2(x); ffn_out = self.ffn(norm_x) ; ffn_out = self.dropout2(ffn_out, deterministic=deterministic)
        x = x + ffn_out # residual connection 
        return x 





In [9]:
def test_encoder_block() -> nn.Module:
    B:int = 32 
    L:int = 128 
    d_model:int = 512 
    n_heads:int = 8 
    d_ffn:int = 2048 # Standard expansion factor of 4x from 512
    dropout_rate:float = 0.1 

    # initialize NNX random state manager with specific hardware streams 
    # 'params' governs weight initialisation 
    # 'dropout' governs the stochastic masking in dropout
    rngs = nn.Rngs(params=42, dropout=42) 
    
    # initialise the encoder block 
    encoder_block = EncoderBlock(d_model=d_model, n_heads=n_heads, d_ffn=d_ffn, dropout_rate=dropout_rate, rngs=rngs, dtype=jnp.bfloat16)

    # creating dummy input simulating the output of embedding layer 
    key = jax.random.PRNGKey(0) 
    dummy_x = jax.random.uniform(key, (B, L, d_model), dtype=jnp.bfloat16)

    print("--- Encoder Block Forward Pass ---")
    print(f"Input shape (B, L, d_model): {dummy_x.shape} | dtype: {dummy_x.dtype}")
    
    # 5. Forward pass (Training mode: deterministic=False)
    output_train = encoder_block(dummy_x, deterministic=False)
    
    print(f"Output shape (B, L, d_model): {output_train.shape} | dtype: {output_train.dtype}")
    
    assert output_train.shape == dummy_x.shape, "Output geometry must match input geometry"
    assert output_train.dtype == jnp.bfloat16, "Precision leaked during compute"
    print("Encoder block passed dimensional and precision invariants."); return encoder_block


In [10]:
model=test_encoder_block()

--- Encoder Block Forward Pass ---
Input shape (B, L, d_model): (32, 128, 512) | dtype: bfloat16
Output shape (B, L, d_model): (32, 128, 512) | dtype: bfloat16
Encoder block passed dimensional and precision invariants.


In [11]:
nn.display(model)

In [12]:
state = nn.state(model, nn.Param) # returns state dict with only Param leaves
num_params = sum(leaf.size for leaf in jax.tree_util.tree_leaves(state)) 
num_bytes = sum(leaf.nbytes for leaf in jax.tree_util.tree_leaves(state))
print(f"Encoder Block:\nTotal trainable parameters: {num_params:,}")
print(f"Memory size ≈ {num_bytes / 1e6:.2f} MiB (float32)")

Encoder Block:
Total trainable parameters: 3,150,336
Memory size ≈ 6.30 MiB (float32)


In [13]:
# positional embedding layer 
from typing import Any


class PositionalEncoding(nn.Module):
    """
    Injects temporal inductive bias into the embedding vectors via deterministic multi scale fourier frequencies 
    Pre-computes the encoding matrix to eliminate forward pass trigonometric FLOP overhead 
    """
    def __init__(self, d_model:int, max_len:int=5_000, dtype:jnp.dtype=jnp.bfloat16) -> None:
        super().__init__()

        # precomputes the structural tensor in float32 for high precision math 
        pe = jnp.zeros((max_len, d_model), dtype=jnp.float32)  
        # positional indices : (max_len, 1)
        position = jnp.arange(0, max_len, dtype=jnp.float32)[:,jnp.newaxis] 
        # Frequency Exponential scaling 
        div_term = jnp.exp(jnp.arange(0, d_model, 2, dtype=jnp.float32) * -(jnp.log(10000.0)/d_model))
        # Apply sin to even indices and cos to odd indices using JAX .at sytax 
        pe = pe.at[:, 0::2].set(jnp.sin(position*div_term))
        pe = pe.at[:, 1::2].set(jnp.cos(position*div_term))
        # Expand batch dimension for XLA broadcasting then cast down to bfloat16 to save bandwidth 
        self.pe_cache = pe[jnp.newaxis,:, :].astype(dtype=dtype) # (1, max_len, d_model)

    def __call__(self, x:jax.Array) -> jax.Array:
        # x shape: (batch, seqLength, d_model) 
        seq_len=x.shape[1]
        # slice the cache upto seq_len and brodcast 
        return x + self.pe_cache[:, :seq_len, :]
    

class TransformerEmbedding(nn.Module): 
    """
    Combines discrete token lookup with continuos positional embedding using embedding lookup table
    """
    def __init__(self, vocab_size:int, d_model:int, max_seq_len:int, dropout_rate:float, rngs:nn.Rngs, dtype:jnp.dtype=jnp.bfloat16): 
        super().__init__() 
        self.d_model = d_model 

        self.token_embed = nn.Embed(num_embeddings=vocab_size, features=d_model, param_dtype=dtype, dtype=dtype, rngs=rngs) # our lookup table 
        self.positional_encoding = PositionalEncoding(d_model=d_model, max_len=max_seq_len, dtype=dtype)
        self.dropout = nn.Dropout(rate=dropout_rate, rngs=rngs) 
    
    def __call__(self, x: jax.Array, deterministic:bool = False) -> jax.Array:
        """
        x : Integer array of shape (batch, seq_len)
        """
        # lookup tokens 
        x_emb = self.token_embed(inputs=x)
        # scale up token embedding to match variance of pos encodings
        # without this, pos signal dominates the semantic signal 
        x_emb = x_emb * jnp.sqrt(jnp.array(self.d_model, dtype=x_emb.dtype)) 
        # Inject temporal inductive bias of positonal trigonometric waves 
        x_pe = self.positional_encoding(x_emb) 
        # Apply stochastic regularisation 
        return self.dropout(x_pe, deterministic=deterministic)


In [14]:
# testing embedding layer 
def test_embedding_layer():
    B:int = 32 
    L:int = 128 
    d_model:int = 512 
    vocab_size:int = 32_000 # Matching the BPE tokeniser vocab we trained 

    # intialise nnx state with params and dropout streams 
    rngs = nn.Rngs(dropout=42, params=42) 

    # instantiate the embedding sub system 
    embed_layer = TransformerEmbedding(vocab_size=vocab_size, d_model=d_model, max_seq_len=1024, dropout_rate=0.1, rngs=rngs, dtype=jnp.bfloat16)
    
    # dummy input of integer token ids simulating jax dataloader output
    key = jax.random.PRNGKey(0) 
    # token ids must strict be integers in range [0, vocab_size-1] 
    dummy_input_ids = jax.random.randint(key, shape=(B, L), minval=0, maxval=vocab_size) 
    print("--- Transformer Embedding Pass ---")
    print(f"Discrete Input shape (B, L): {dummy_input_ids.shape} | dtype: {dummy_input_ids.dtype}")
    
    # Forward pass mapping integers to continuous vectors
    output_embeddings = embed_layer(dummy_input_ids, deterministic=False)
    
    print(f"Continuous Output shape (B, L, d_model): {output_embeddings.shape} | dtype: {output_embeddings.dtype}")
    
    # Assert structural invariants
    assert output_embeddings.shape == (B, L, d_model), "Embedding failed to project to d_model manifold"
    assert output_embeddings.dtype == jnp.bfloat16, "Precision constraint violated"
    print("Embedding projection and symmetry breaking passed.")

In [15]:
test_embedding_layer()

--- Transformer Embedding Pass ---
Discrete Input shape (B, L): (32, 128) | dtype: int32
Continuous Output shape (B, L, d_model): (32, 128, 512) | dtype: bfloat16
Embedding projection and symmetry breaking passed.


In [16]:
# tranformer encoder block 
class TransformerEncoder(nn.Module):
    """
    Complete N layer transformer encoder stack.
    Maps discrete (B,L) token indices to continuous (B, L, d_model) contextual representations
    """
    def __init__(self, vocab_size:int, d_model:int, n_heads:int, d_ff:int, n_layers:int, max_len:int, dropout_rate:float, rngs:nn.Rngs, type:jnp.dtype=jnp.bfloat16): 
        super().__init__() 

        # Input Embeddings 
        self.embedding = TransformerEmbedding(vocab_size=vocab_size, d_model=d_model, max_seq_len=max_len, dropout_rate=dropout_rate, rngs=rngs, dtype=type) 

        # Deep stack of N independent encoder blocks
        # Using a standard list comprehension cleanly instantiates N distinct parameter states 
        # XLA will unroll this seamlessly during compilation
        self.layers = nn.List([EncoderBlock(d_model=d_model, n_heads=n_heads, d_ffn=d_ff, dropout_rate=dropout_rate, rngs=rngs, dtype=type)
                       for _ in range(n_layers)])
        # Final Layer Norm Projection 
        self.final_norm = nn.LayerNorm(num_features=d_model, dtype=type, param_dtype=type, rngs=rngs)

    def __call__(self, x:jax.Array, padding_mask:jax.Array|None=None, deterministic:bool=False) -> jax.Array:
        """
        x: (batch, seq_length) integer tokens 
        padding_mask: (Batch, 1, 1, seq_length) shape boolean mask to ignore [PAD] tokens
        deterministic: bool = True during inference/eval to disable dropouts
        """
        # project tokens to continuous space + positional encodings 
        hidden_states = self.embedding(x, deterministic=deterministic) 
        # iteratively refine representations through the block sequence 
        for block in self.layers: hidden_states = block(hidden_states, mask=padding_mask, deterministic=deterministic) 
        # Final geometry stabilization
        return self.final_norm(hidden_states)


In [17]:
def test_transformer_encoder():
    # Architecture definitions matching the original paper's base model 
    B:int = 32 
    L:int = 128 
    d_model:int = 512 
    num_heads:int = 8 
    d_ffn:int = 2048 
    num_layers:int = 6 # replicating Vaswani et. al 
    vocab_size:int = 32_000

    rngs = nn.Rngs(params=42, dropout=42) 

    # instantiate the full tower 
    encoder = TransformerEncoder(vocab_size=vocab_size, d_model=d_model, n_heads=num_heads, d_ff=d_ffn, n_layers=num_layers, max_len=1024, dropout_rate=0.1, rngs=rngs, type=jnp.bfloat16)

    # Generate dummy input tokens (batch=32, seq_len=128) 
    key = jax.random.PRNGKey(0) 
    input_ids = jax.random.randint(key, shape=(B, L), minval=0, maxval=vocab_size) 
    
    # PADDING MASK 
    # in our tokeniser 1 is pad token ID 
    # we create a boolean mask where True means "attend" and False means "ignore" 
    # shape needs to broadcast over (Batch, NumHeads, SeqLen, SeqLen) in the attention logic
    pad_id = 1 
    # Shape: (B, 1, 1, L) 
    padding_mask = (input_ids != pad_id)[:, jnp.newaxis, jnp.newaxis, :] 

    print("--- Full Transformer Encoder Pass ---")
    print(f"Discrete Input shape (B, L): {input_ids.shape}")
    print(f"Padding Mask shape: {padding_mask.shape}")
    
    # Execute the deep representation learning
    encoder_output = encoder(input_ids, padding_mask=padding_mask, deterministic=False)
    
    print(f"Contextualized Output shape (B, L, d_model): {encoder_output.shape} | dtype: {encoder_output.dtype}")
    state = nn.state(encoder, nn.Param) # returns state dict with only Param leaves
    num_params = sum(leaf.size for leaf in jax.tree_util.tree_leaves(state)) 
    num_bytes = sum(leaf.nbytes for leaf in jax.tree_util.tree_leaves(state))
    print(f"\nEncoder :\nTotal trainable parameters: {num_params:,}")
    print(f"Number of Encoder Layers: {num_layers}")
    print(f"Memory size ≈ {num_bytes / 1e6:.2f} MiB (float32)")
    
    assert encoder_output.shape == (B, L, d_model), "Encoder tower failed to maintain spatial/channel geometry."
    assert encoder_output.dtype == jnp.bfloat16, "Precision leaked during deep propagation."
    print("Encoder Tower is structurally flawless.")


In [18]:
test_transformer_encoder()

--- Full Transformer Encoder Pass ---
Discrete Input shape (B, L): (32, 128)
Padding Mask shape: (32, 1, 1, 128)
Contextualized Output shape (B, L, d_model): (32, 128, 512) | dtype: bfloat16

Encoder :
Total trainable parameters: 35,287,040
Number of Encoder Layers: 6
Memory size ≈ 70.57 MiB (float32)
Encoder Tower is structurally flawless.


In [19]:
import jax
import jax.numpy as jnp
from flax import nnx
import time

# ---------------------------------------------------------
# 1. Unfused Architecture
# ---------------------------------------------------------
class UnfusedQKV(nnx.Module):
    def __init__(self, d_model: int, rngs: nnx.Rngs):
        # 3 separate weight matrices loaded into SRAM sequentially
        self.w_q = nnx.Linear(d_model, d_model, rngs=rngs)
        self.w_k = nnx.Linear(d_model, d_model, rngs=rngs)
        self.w_v = nnx.Linear(d_model, d_model, rngs=rngs)

    def __call__(self, x: jax.Array):
        q = self.w_q(x)
        k = self.w_k(x)
        v = self.w_v(x)
        return q, k, v

# ---------------------------------------------------------
# 2. Fused Architecture
# ---------------------------------------------------------
class FusedQKV(nnx.Module):
    def __init__(self, d_model: int, rngs: nnx.Rngs):
        # 1 contiguous weight matrix loaded into SRAM once
        self.w_qkv = nnx.Linear(d_model, 3 * d_model, rngs=rngs)
        self.d_model = d_model

    def __call__(self, x: jax.Array):
        qkv = self.w_qkv(x)
        # Split along the feature dimension to retrieve Q, K, V
        q, k, v = jnp.split(qkv, 3, axis=-1)
        return q, k, v

# ---------------------------------------------------------
# 3. Micro-Benchmarking Setup
# ---------------------------------------------------------
def benchmark():
    # Model parameters
    d_model = 4096
    batch_size = 1 # Simulating autoregressive generation (token-by-token)
    seq_len = 1
    
    # Initialize random states
    rngs = nnx.Rngs(0)
    x = jax.random.normal(jax.random.PRNGKey(42), (batch_size, seq_len, d_model))

    # Instantiate NNX Modules
    unfused_model = UnfusedQKV(d_model, rngs=rngs)
    fused_model = FusedQKV(d_model, rngs=rngs)

    # Compile forward passes
    @nnx.jit
    def forward_unfused(model, x):
        return model(x)

    @nnx.jit
    def forward_fused(model, x):
        return model(x)

    # Warmup phase (forces XLA compilation)
    print("Compiling and warming up...")
    _ = forward_unfused(unfused_model, x)
    _ = forward_fused(fused_model, x)
    
    # Ensure all asynchronous execution is finished before timing
    jax.block_until_ready(_)

    iterations = 1000

    # Benchmark Unfused
    start_time = time.time()
    for _ in range(iterations):
        _ = forward_unfused(unfused_model, x)
    jax.block_until_ready(_) # Block CPU until GPU finishes
    unfused_time = time.time() - start_time

    # Benchmark Fused
    start_time = time.time()
    for _ in range(iterations):
        _ = forward_fused(fused_model, x)
    jax.block_until_ready(_)
    fused_time = time.time() - start_time

    print(f"Unfused Latency (1k runs): {unfused_time:.4f} seconds")
    print(f"Fused Latency (1k runs):   {fused_time:.4f} seconds")

if __name__ == "__main__":
    benchmark()

Compiling and warming up...
Unfused Latency (1k runs): 1.8411 seconds
Fused Latency (1k runs):   1.8538 seconds


In [20]:
class CausalMultiHeadAttention(nn.Module): 
    """ 
    Multi-head causal self attention used in the decoder
    This forces autoregressive (left-to-right) generation by masking future positions. 

    Compared to encoder attention:
    - Always applies a causal (lower-triangular mask)
    - Usually called with same sequence as Q=K=V (self-attention)
    """

    def __init__(self, d_model:int, num_heads:int, rngs:nn.Rngs, dropout_rate:float=0.0, dtype: jnp.dtype= jnp.bfloat16):
        super().__init__()

        self.mha = MultiHeadSelfAttention(d_model=d_model, num_heads=num_heads, rngs=rngs, dropout_rate=dropout_rate, dtype=dtype)
        self.num_heads=num_heads
        self.dtype=dtype

    def __call__(self, x: jax.Array, padding_mask:jax.Array|None=None, deterministic:bool=False) -> jax.Array:
        """ 
        Args:
            x : Query, key and value all come from the same tensor of shape(batch, target_seq_len, d_model) 
            padding_mask: Optinal padding mask (usually from target side) of shape(batch, 1, 1, target_seq_len) - true = attend 
            deterministic: If True, disable dropout

        Returns: 
            Output of shape (batch, target_seq_Len, d_model)
        """

        B, L_dec, d_model = x.shape
        causal_mask = jnp.tril(jnp.ones((L_dec, L_dec), dtype=jnp.bool_))

        # reshape for broadcast -> (1, 1, L_dec, L_dec)
        causal_mask = causal_mask[None, None, :, :] # this adds two extra dims 

        # combine padding mask and causal mask 
        if padding_mask is not None:
            # padding mask : (B, 1, 1 , L_dec) 
            # causal mask : (1, 1, L_dec, L_dec) 
            # combined shape : (B, 1, L_dec, L_dec) 

            # True -> can attend 
            # False -> Cannot attend 
            combined_mask = padding_mask & causal_mask # logical AND operation 

        else: combined_mask = causal_mask 

        # use mha to pass in mask as a parameter 
        return self.mha(x=x, mask=combined_mask, deterministic=deterministic)

In [21]:
# function to test causal multihead attention mechanism 
def test_causal_multihead_attention(): 
    """ 
    Small standalone function which tests causal multi head attention 
    used in decoder block of the transformer model. 
    This testing func, checks dtype consistencies, device placement and basic masking behavior
    """
    # hyperparameters 
    B:int = 16 # batch size for parallel comp
    L:int = 128  # max_seq_len (decoder) in our case same as encoder
    d_model:int = 512 # replicating attention paper 
    num_heads:int = 8 # from paper 
    dropout_rate:float = 0.1 

    rngs=nn.Rngs(params=744, dropout=744) 

    causal_mha = CausalMultiHeadAttention(d_model=d_model, num_heads=num_heads, rngs=rngs, dtype=jnp.bfloat16) 

    # creating dummy input to simulate encoder output (input for decoder) 
    key = jax.random.PRNGKey(744) 
    dummy_x = jax.random.normal(key, shape=(B, L, d_model), dtype=jnp.bfloat16) 
    
    # padding mask here 
    pad_tok_fraction:float = 0.15 # 15% tokens 
    padding_mask = jax.random.bernoulli(key, p=1.0-pad_tok_fraction, shape=(B,L))[:, None, None, :] # (B, 1, 1, L) -> True = attend

    print("=== Causal Multi-Head Attention Test ===")
    print(f"Input shape              : {dummy_x.shape} | dtype: {dummy_x.dtype}")
    print(f"Padding mask shape       : {padding_mask.shape}")
    print(f"Model dtype              : {causal_mha.mha.dtype}")
    print(f"Number of heads          : {num_heads}")
    print(f"d_k (head dimension)     : {d_model // num_heads}")

    #forward pass (dropout active ) 
    output_train = causal_mha(dummy_x, padding_mask=padding_mask, deterministic=False) 

    # forward pass - inference mode (dropout off ) 
    output_infer = causal_mha(dummy_x, padding_mask=padding_mask, deterministic=True) 

    # ─── Basic shape and type checks ──────────────────────────────────────────
    print("\nResults:")
    print(f"Output shape (train)     : {output_train.shape} | dtype: {output_train.dtype}")
    print(f"Output shape (infer)     : {output_infer.shape} | dtype: {output_infer.dtype}")

    assert output_train.shape == (B, L, d_model), "Shape mismatch in training mode"
    assert output_infer.shape == (B, L, d_model), "Shape mismatch in inference mode"
    assert output_train.dtype == jnp.bfloat16, "Dtype leak in training mode"
    assert output_infer.dtype == jnp.bfloat16, "Dtype leak in inference mode"

    # ─── Very basic correctness check: outputs should be different due to dropout ─
    diff_norm = jnp.mean(jnp.abs(output_train - output_infer))
    print(f"Mean absolute difference (train vs infer): {float(diff_norm):.6f}")
    print("(should be > 0 because dropout is active in train mode)")

    # ─── Count parameters ----
    state = nn.state(causal_mha, nn.Param)
    num_params = sum(leaf.size for leaf in jax.tree_util.tree_leaves(state))
    num_bytes  = sum(leaf.nbytes for leaf in jax.tree_util.tree_leaves(state))

    print(f"\nCausal MHA module parameters : {num_params:,}")
    print(f"Approx memory (bf16)         : {num_bytes / 1e6:.2f} MiB")

    print("\n→ All basic invariants passed.")
    print("   (shape, dtype, dropout behavior, parameter count)")

In [22]:
prettify = lambda: print("*" * 50)
print("Testing Causal Multi-Head Attention"); test_causal_multihead_attention(); prettify()

Testing Causal Multi-Head Attention
=== Causal Multi-Head Attention Test ===
Input shape              : (16, 128, 512) | dtype: bfloat16
Padding mask shape       : (16, 1, 1, 128)
Model dtype              : <class 'jax.numpy.bfloat16'>
Number of heads          : 8
d_k (head dimension)     : 64

Results:
Output shape (train)     : (16, 128, 512) | dtype: bfloat16
Output shape (infer)     : (16, 128, 512) | dtype: bfloat16
Mean absolute difference (train vs infer): 0.000000
(should be > 0 because dropout is active in train mode)

Causal MHA module parameters : 1,048,576
Approx memory (bf16)         : 2.10 MiB

→ All basic invariants passed.
   (shape, dtype, dropout behavior, parameter count)
**************************************************


In [23]:
class CrossAttention(nn.Module):
    """
    Cross-attention module for the Transformer decoder.
    Allows the decoder to attend to the encoder's output for alignment.
    """
    def __init__(self, d_model:int, num_heads:int, rngs:nn.Rngs, dropout_rate:float=0.0, dtype:jnp.dtype=jnp.bfloat16):
        super().__init__()
        if d_model % num_heads != 0: raise ValueError(f"d_model ({d_model}) must be divisible by num_heads ({num_heads})")
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.dtype = dtype
        self.scale = jax.lax.rsqrt(jnp.array(self.d_k, dtype=self.dtype))

        # Separate projections for Queries (Decoder) and Keys/Values (Encoder)
        self.q_proj = nn.Linear(in_features=self.d_model, out_features=self.d_model, use_bias=False, dtype=self.dtype, param_dtype=self.dtype, rngs=rngs)
        self.kv_proj = nn.Linear(in_features=self.d_model, out_features=2 * self.d_model, use_bias=False, dtype=self.dtype, param_dtype=self.dtype, rngs=rngs)
        self.out_proj = nn.Linear(in_features=self.d_model, out_features=self.d_model, use_bias=False, dtype=self.dtype, param_dtype=self.dtype, rngs=rngs)
        self.dropout = nn.Dropout(rate=dropout_rate, rngs=rngs)

    def __call__(self, x:jax.Array, context:jax.Array, mask:jax.Array|None=None, deterministic:bool=False) -> jax.Array:
        """
        x: Decoder queries, shape (B, target_seq_len, d_model)
        context: Encoder output, shape (B, source_seq_len, d_model)
        mask: Optional padding mask from encoder, shape (B, 1, 1, source_seq_len)
        """
        B, L_q, _ = x.shape
        _, L_kv, _ = context.shape

        q = self.q_proj(x) # (B, L_q, d_model)
        kv = self.kv_proj(context) # (B, L_kv, 2 * d_model)

        q = q.reshape(B, L_q, self.num_heads, self.d_k)
        kv = kv.reshape(B, L_kv, 2, self.num_heads, self.d_k)

        # transpose: (B, num_heads, L_q, d_k)
        q = jnp.transpose(q, (0, 2, 1, 3))
        # isolate k, v, transpose: (B, num_heads, L_kv, d_k)
        kv = jnp.transpose(kv, (2, 0, 3, 1, 4))
        k, v = kv[0], kv[1]

        # Scaled dot product attention
        logits = jnp.einsum('bhqd, bhkd -> bhqk', q, k) * self.scale

        if mask is not None:
            neg_inf = jnp.array(-1e9, dtype=logits.dtype)
            logits = jnp.where(mask, logits, neg_inf)
        
        attention_weights = jax.nn.softmax(logits.astype(jnp.float32), axis=-1).astype(self.dtype)
        attention_weights = self.dropout(attention_weights, deterministic=deterministic)

        out = jnp.einsum('bhqk, bhkd -> bhqd', attention_weights, v)
        out = jnp.transpose(out, (0, 2, 1, 3)).reshape(B, L_q, self.d_model)

        return self.out_proj(out)

In [24]:
# function to test causal multihead attention mechanism 
def test_causal_multihead_attention(): 
    """ 
    Small standalone function which tests causal multi head attention 
    used in decoder block of the transformer model. 
    This testing func, checks dtype consistencies, device placement and basic masking behavior
    """
    # hyperparameters 
    B:int = 16 # batch size for parallel comp
    L:int = 128  # max_seq_len (decoder) in our case same as encoder
    d_model:int = 512 # replicating attention paper 
    num_heads:int = 8 # from paper 
    dropout_rate:float = 0.1 

    rngs=nn.Rngs(params=744, dropout=744) 

    causal_mha = CausalMultiHeadAttention(d_model=d_model, num_heads=num_heads, rngs=rngs, dropout_rate=dropout_rate, dtype=jnp.bfloat16) 

    # creating dummy input to simulate encoder output (input for decoder) 
    key = jax.random.PRNGKey(744) 
    dummy_x = jax.random.normal(key, shape=(B, L, d_model), dtype=jnp.bfloat16) 
    
    # padding mask here 
    pad_tok_fraction:float = 0.15 # 15% tokens 
    padding_mask = jax.random.bernoulli(key, p=1.0-pad_tok_fraction, shape=(B,L))[:, None, None, :] # (B, 1, 1, L) -> True = attend

    print("=== Causal Multi-Head Attention Test ===")
    print(f"Input shape              : {dummy_x.shape} | dtype: {dummy_x.dtype}")
    print(f"Padding mask shape       : {padding_mask.shape}")
    print(f"Model dtype              : {causal_mha.mha.dtype}")
    print(f"Number of heads          : {num_heads}")
    print(f"d_k (head dimension)     : {d_model // num_heads}")

    #forward pass (dropout active ) 
    output_train = causal_mha(dummy_x, padding_mask=padding_mask, deterministic=False) 

    # forward pass - inference mode (dropout off ) 
    output_infer = causal_mha(dummy_x, padding_mask=padding_mask, deterministic=True) 

    # ─── Basic shape and type checks ──────────────────────────────────────────
    print("\nResults:")
    print(f"Output shape (train)     : {output_train.shape} | dtype: {output_train.dtype}")
    print(f"Output shape (infer)     : {output_infer.shape} | dtype: {output_infer.dtype}")

    assert output_train.shape == (B, L, d_model), "Shape mismatch in training mode"
    assert output_infer.shape == (B, L, d_model), "Shape mismatch in inference mode"
    assert output_train.dtype == jnp.bfloat16, "Dtype leak in training mode"
    assert output_infer.dtype == jnp.bfloat16, "Dtype leak in inference mode"

    # ─── Very basic correctness check: outputs should be different due to dropout ─
    diff_norm = jnp.mean(jnp.abs(output_train - output_infer))
    print(f"Mean absolute difference (train vs infer): {float(diff_norm):.6f}")
    print("(should be > 0 because dropout is active in train mode)")

    # ─── Count parameters ----
    state = nn.state(causal_mha, nn.Param)
    num_params = sum(leaf.size for leaf in jax.tree_util.tree_leaves(state))
    num_bytes  = sum(leaf.nbytes for leaf in jax.tree_util.tree_leaves(state))

    print(f"\nCausal MHA module parameters : {num_params:,}")
    print(f"Approx memory (bf16)         : {num_bytes / 1e6:.2f} MiB")

    print("\n→ All basic invariants passed.")
    print("   (shape, dtype, dropout behavior, parameter count)")


In [25]:
# implementing decoder block here 
class DecoderBlock(nn.Module):
    """
    Single block of transformer decoder. 
    Final block will have 6 of these. 
    
    Features (from bottom to top in the architecture diagram):
        1. Masked Multi-Head Attention: Attends to previous tokens in the target sequence (prevents looking ahead).
        2. Cross Attention: Attends to the encoder's output to ground the generation in the source sentence.
        3. Feed Forward: Expands and contracts dimensions for non-linear feature transformation.
        
    Note: Like the EncoderBlock, we use pre-LayerNorm which applies normalization BEFORE each sub-layer. 
    This is known to be more stable for training deep transformers than the original paper's post-LayerNorm.
    """
    def __init__(self, d_model:int, n_heads:int, d_ffn:int, dropout_rate:float, rngs:nn.Rngs, dtype:jnp.dtype=jnp.bfloat16): 
        super().__init__()
        
        # Sub-layer 1: Masked Multi-Head Self Attention
        self.masked_mha = CausalMultiHeadAttention(d_model=d_model, num_heads=n_heads, rngs=rngs, dropout_rate=dropout_rate, dtype=dtype)
        self.ln1 = nn.LayerNorm(num_features=d_model, dtype=dtype, param_dtype=dtype, rngs=rngs)
        self.dropout1 = nn.Dropout(rate=dropout_rate, rngs=rngs)
        
        # Sub-layer 2: Cross Attention (Decoder-Encoder Attention)
        self.cross_mha = CrossAttention(d_model=d_model, num_heads=n_heads, rngs=rngs, dropout_rate=dropout_rate, dtype=dtype)
        self.ln2 = nn.LayerNorm(num_features=d_model, dtype=dtype, param_dtype=dtype, rngs=rngs)
        self.dropout2 = nn.Dropout(rate=dropout_rate, rngs=rngs)
        
        # Sub-layer 3: Position-wise Feed Forward Network
        self.ffn = PositionWiseFFN(d_model=d_model, d_ffn=d_ffn, rngs=rngs, dtype=dtype)
        self.ln3 = nn.LayerNorm(num_features=d_model, dtype=dtype, param_dtype=dtype, rngs=rngs)
        self.dropout3 = nn.Dropout(rate=dropout_rate, rngs=rngs)

    def __call__(self, x:jax.Array, encoder_output:jax.Array, target_padding_mask:jax.Array|None=None, source_padding_mask:jax.Array|None=None, deterministic:bool=False) -> jax.Array:
        """
        x: Target sequence embeddings/previous layer output -> Shape: (Batch, Target_SeqLen, d_model)
        encoder_output: Output from the top of the encoder stack -> Shape: (Batch, Source_SeqLen, d_model)
        target_padding_mask: Mask for target sequence (e.g., to ignore <PAD> tokens) -> Shape: (Batch, 1, 1, Target_SeqLen)
        source_padding_mask: Mask for source sequence (to prevent attending to source <PAD> tokens in cross-attention) -> Shape: (Batch, 1, 1, Source_SeqLen)
        deterministic: True during inference/eval to disable dropout.
        """
        
        # ---------------------------------------------------------
        # Sub-layer 1: Masked Multi-Head Self-Attention
        # Purpose: Allow the decoder to look at previous words in the target sequence it has generated so far.
        # The true causal mask (lower triangular) is handled inside the CausalMultiHeadAttention module.
        # ---------------------------------------------------------
        norm_x = self.ln1(x) 
        attn_out1 = self.masked_mha(norm_x, padding_mask=target_padding_mask, deterministic=deterministic) 
        attn_out1 = self.dropout1(attn_out1, deterministic=deterministic)
        x = x + attn_out1 # Residual connection 1: Add & Norm (Note: Norm was applied to input, so this is pre-LN residual)
        
        # ---------------------------------------------------------
        # Sub-layer 2: Cross-Attention
        # Purpose: Allow the decoder to focus on relevant parts of the input sequence (encoder_output).
        # Query (Q) comes from the previous decoder layer (x).
        # Key (K) and Value (V) come from the encoder's output.
        # ---------------------------------------------------------
        norm_x = self.ln2(x)
        # In Cross Attention: x is Query (target), encoder_output is Key/Value (source)
        attn_out2 = self.cross_mha(x=norm_x, context=encoder_output, mask=source_padding_mask, deterministic=deterministic)
        attn_out2 = self.dropout2(attn_out2, deterministic=deterministic)
        x = x + attn_out2 # Residual connection 2
        
        # ---------------------------------------------------------
        # Sub-layer 3: Position-wise Feed Forward
        # Purpose: Apply additional non-linear transformations to each position independently.
        # ---------------------------------------------------------
        norm_x = self.ln3(x)
        ffn_out = self.ffn(norm_x)
        ffn_out = self.dropout3(ffn_out, deterministic=deterministic)
        x = x + ffn_out # Residual connection 3
        
        # Returns the updated representation for the target sequence
        # Shape: (Batch, Target_SeqLen, d_model)
        return x

In [26]:
def test_decoder_block() -> nn.Module:
    # Hyperparameters
    B:int = 32               # Batch size
    L_target:int = 64        # Sequence length of target (decoder)
    L_source:int = 128       # Sequence length of source (encoder)
    d_model:int = 512        # Dimensionality of the model
    n_heads:int = 8          # Number of attention heads
    d_ffn:int = 2048         # Dimensionality of feed-forward network
    dropout_rate:float = 0.1 # Standard dropout

    # Initialize RNgs
    rngs = nn.Rngs(params=42, dropout=42) 
    
    # Initialize the Decoder Block
    decoder_block = DecoderBlock(d_model=d_model, n_heads=n_heads, d_ffn=d_ffn, dropout_rate=dropout_rate, rngs=rngs, dtype=jnp.bfloat16)

    # Setup dummy data for inputs
    key = jax.random.PRNGKey(0) 
    key1, key2, key3, key4 = jax.random.split(key, 4)
    
    # 1. Dummy target sequence (Query for cross attention, input for masked MHA)
    dummy_target = jax.random.uniform(key1, (B, L_target, d_model), dtype=jnp.bfloat16)
    # 2. Dummy encoder output (Keys/Values for cross attention)
    dummy_encoder_out = jax.random.uniform(key2, (B, L_source, d_model), dtype=jnp.bfloat16)

    # 3. Simulate Padding masks (True = process, False = ignore padding)
    target_pad_fraction = 0.15
    target_padding_mask = jax.random.bernoulli(key3, p=1.0-target_pad_fraction, shape=(B, L_target))[:, None, None, :]
    
    source_pad_fraction = 0.15
    source_padding_mask = jax.random.bernoulli(key4, p=1.0-source_pad_fraction, shape=(B, L_source))[:, None, None, :]

    print("--- Decoder Block Forward Pass ---")
    print(f"Target Input shape (B, L_tar, d_model): {dummy_target.shape} | dtype: {dummy_target.dtype}")
    print(f"Encoder Output shape (B, L_src, d_model): {dummy_encoder_out.shape}")
    print(f"Target Padding mask shape: {target_padding_mask.shape}")
    print(f"Source Padding mask shape: {source_padding_mask.shape}")
    
    # Execute Forward Pass
    output_train = decoder_block(x=dummy_target, 
                                 encoder_output=dummy_encoder_out, 
                                 target_padding_mask=target_padding_mask, 
                                 source_padding_mask=source_padding_mask, 
                                 deterministic=False)
    
    print(f"\nOutput shape (B, L_tar, d_model): {output_train.shape} | dtype: {output_train.dtype}")
    
    # Run assertions
    assert output_train.shape == dummy_target.shape, "Output geometry must match input target geometry"
    assert output_train.dtype == jnp.bfloat16, "Precision leaked during compute"
    
    # Parameter count
    state = nn.state(decoder_block, nn.Param)
    num_params = sum(leaf.size for leaf in jax.tree_util.tree_leaves(state))
    print(f"Decoder Block | Total trainable parameters: {num_params:,}")
    print("→ Decoder block passed dimensional and precision invariants.")
    
    return decoder_block

# Run it
model = test_decoder_block()


--- Decoder Block Forward Pass ---
Target Input shape (B, L_tar, d_model): (32, 64, 512) | dtype: bfloat16
Encoder Output shape (B, L_src, d_model): (32, 128, 512)
Target Padding mask shape: (32, 1, 1, 64)
Source Padding mask shape: (32, 1, 1, 128)

Output shape (B, L_tar, d_model): (32, 64, 512) | dtype: bfloat16
Decoder Block | Total trainable parameters: 4,199,936
→ Decoder block passed dimensional and precision invariants.
